<a href="https://colab.research.google.com/github/fursew05/Dacon/blob/main/1%EB%8B%A8%EA%B3%84(%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%88%98%EC%A7%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 재무 데이터 수집

- TS2000에서 코스닥, 외감 기업의 2010년 ~ 2023년 연결IFRS, 개별IFRS, GAAP 재무 데이터 수집
- 12월 결산 법인만 수집

In [ ]:
import pandas as pd
import re
import os
from pathlib import Path
import numpy as np

pd.set_option("display.max_columns", None)   # 열 생략 없이 전부 출력
pd.set_option("display.expand_frame_repr", False)  # 줄 넘김 없이 한 줄에 출력

In [ ]:
# 코스닥 데이터 불러오기
kosdaq_연결IFRS = pd.read_excel('../data/kosdaq_연결재무비율(IFRS).xlsx')
kosdaq_개별IFRS = pd.read_excel('../data/kosdaq_개별재무비율(IFRS).xlsx')
kosdaq_gaap = pd.read_excel('../data/kosdaq_재무비율(GAAP).xlsx')

dfs = {
    'kosdaq_연결IFRS': kosdaq_연결IFRS,
    'kosdaq_개별IFRS': kosdaq_개별IFRS,
    'kosdaq_gaap': kosdaq_gaap,
}

for name, df in dfs.items():
    print(f'=== {name}의 컬럼 수 ===')
    # 3-1. 기본 정보: 행·열 개수, 컬럼별 non-null 값, 데이터 타입
    print(len(df.columns))
    print(f'=== {name}의 데이터 수 ===')
    print(len(df))

In [ ]:
# 비상장 외감 데이터 불러오기
out_gaap_1 = pd.read_excel('../data/외감_재무_page1.xlsx')
out_gaap_2 = pd.read_excel('../data/외감_재무_page2.xlsx')
out_gaap_3 = pd.read_excel('../data/외감_재무_page3.xlsx')
out_gaap_4 = pd.read_excel('../data/외감_재무_page4.xlsx')

dfs = {
    'out_gaap_1': out_gaap_1,
    'out_gaap_2': out_gaap_2,
    'out_gaap_3': out_gaap_3,
    'out_gaap_4': out_gaap_4
}

for name, df in dfs.items():
    print(f'=== {name}의 컬럼 수 ===')
    # 3-1. 기본 정보: 행·열 개수, 컬럼별 non-null 값, 데이터 타입
    print(len(df.columns))
    print(f'=== {name}의 데이터 수 ===')
    print(len(df))

## 데이터 통합 (컬럼 이름 변경)
- 의미는 같지만 이름이 다른 컬럼들 통일

In [ ]:
# --- 1) 컬럼명 클린 함수 -------------------------------------------------
pattern = r'\(IFRS(?:연결)?\)'
def clean_columns(df):
    df = df.copy()
    df.columns = [re.sub(pattern, '', col).strip() for col in df.columns]
    return df

# --- 2) 모든 DataFrame 클린 --------------------------------------------
dfs_clean = {
    name: clean_columns(df)
    for name, df in {
        'kosdaq_연결IFRS': kosdaq_연결IFRS,
        'kosdaq_개별IFRS': kosdaq_개별IFRS,
        'kosdaq_gaap': kosdaq_gaap,
        'out_gaap_1': out_gaap_1,
        'out_gaap_2': out_gaap_2,
        'out_gaap_3': out_gaap_3,
        'out_gaap_4': out_gaap_4
    }.items()
}

# --- 3) out_gaap 컬럼 집합(클린된 상태) -----------------------------------
out_cols = set(dfs_clean['out_gaap_1'].columns)

# --- 4) 의미는 같지만 이름이 다른 컬럼 매핑 ------------------------------
rename_map = {
    # EPS
    '1주당순이익': 'EPS', 'EPS(Earning Per Share)': 'EPS',
    # BPS
    '1주당순자산': 'BPS', 'BPS(Book-value Per Share)': 'BPS',
    # CPS
    '1주당CASH FLOW': 'CPS', 'CPS(Cash flow Per Share)': 'CPS',
    # OPPS
    '1주당정상영업이익': 'OPPS', '1주당영업이익': 'OPPS',
    # 이자보상배율
    '정상영업이익대비이자보상배율': '이자보상배율(영업이익)',
    '영업이익대비이자보상배율':   '이자보상배율(영업이익)',
    # 영업이익률
    '매출액정상영업이익률': '영업이익률(매출액)', '매출액영업이익률': '영업이익률(매출액)',
    # 영업이익증가율
    '정상영업이익증가율': '영업이익증가율', '영업이익증가율': '영업이익증가율',
    # 종업원당영업이익
    '종업원1인당 정상영업이익(백만원)':            '종업원당영업이익(백만원)',
    '종업원1인당 경상이익 (2007년 이전 발생)(백만원)': '종업원당영업이익(백만원)',
    # ROE / ROC / ROA
    '자기자본정상영업이익률':  '자기자본영업이익률', '자기자본영업이익률': '자기자본영업이익률',
    '자본금정상영업이익률':    '자본금영업이익률',   '자본금영업이익률':   '자본금영업이익률',
    '총자본정상영업이익률':    '총자본영업이익률',   '총자본영업이익률':   '총자본영업이익률',
    # 영업현금흐름
    '영업활동으로 인한 현금흐름(간접법)(*)(천원)': '영업현금흐름(천원)',
    '영업활동으로 인한 현금흐름(*)(천원)':        '영업현금흐름(천원)',
    # 세전이익대비세금율 / 총비용대비세금율 / 세금금액
    '세금과공과 대 세금과공과 차감전순이익률': '세전이익대비세금율(차감전)',
    '조세 대 조세차감전순이익률':           '세전이익대비세금율(차감전)',
    '세금과공과 대 총비용비율':            '총비용대비세금율',
    '조세공과 대 총비용비율':             '총비용대비세금율',
    '세금과공과(백만원)':                '세금금액(백만원)',
    '조세공과(백만원)':                  '세금금액(백만원)',
    # 영업비용대영업수익비율
    '영업비용 대 영업수익비율': '영업비용대영업수익비율',
    '영업비율':               '영업비용대영업수익비율',
    # 비영업손익률
    '기타손익비률': '비영업손익률(기타)', '금융손익비율': '비영업손익률(금융)', '영업외손익률': '비영업손익률',
    # 수지비율
    '수지비율(관계기업투자손익 제외)': '수지비율', '수지비율': '수지비율',
    # 경영자본영업이익률
    '경영자본정상영업이익률': '경영자본영업이익률',
    '경영자본영업이익률':     '경영자본영업이익률'
}

# --- 5) IFRS·연결IFRS : out_gaap과만 다른 컬럼에 대해 rename --------------
for name in ['kosdaq_개별IFRS', 'kosdaq_연결IFRS']:
    df = dfs_clean[name]
    unique_non_overlap = set(df.columns) - out_cols         # out_gaap과 겹치지 않는 컬럼
    sub_map = {k: v for k, v in rename_map.items() if k in unique_non_overlap}
    dfs_clean[name] = df.rename(columns=sub_map)

# --- 6) NEW ― kosdaq_gaap & out_gaap 도 동일 규칙으로 rename -------------
for name in ['kosdaq_gaap', 'out_gaap_1','out_gaap_2','out_gaap_3','out_gaap_4']:
    df = dfs_clean[name]
    sub_map = {k: v for k, v in rename_map.items() if k in df.columns}
    df = df.rename(columns=sub_map)
    # 같은 이름으로 합쳐져 중복된 컬럼은 첫 번째만 남김
    df = df.loc[:, ~df.columns.duplicated()]
    dfs_clean[name] = df

# --- 7) 결과 확인 ---------------------------------------------------------
for name, df in dfs_clean.items():
    print(f"{name} ({len(df.columns)} cols):")
    print(df.columns.tolist(), "\n")


kosdaq_연결IFRS (141 cols):
['회사명', '거래소코드', '회계년도', '총자본증가율', '유형자산증가율', '비유동생물자산증가율', '투자부동산증가율', '비유동자산증가율', '유동자산증가율', '재고자산증가율', '자기자본증가율', '매출액증가율', '영업이익증가율', '순이익증가율', '총포괄이익증가율', '종업원1인당 부가가치증가율', '종업원수증가율', '종업원1인당 매출액증가율', '종업원1인당 인건비증가율', '매출액총이익률', '영업이익률(매출액)', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '총자본순이익률', '자기자본영업이익률', '자기자본순이익률', '경영자본영업이익률', '경영자본순이익률', '자본금영업이익률', '자본금순이익률', '매출원가 대 매출액비율', '영업비용대영업수익비율', '비영업손익률(기타)', '비영업손익률(금융)', '금융비용부담률', '외환이익 대 매출액비율', '광고선전비 대 매출액비율', '세전이익대비세금율(차감전)', '기업순이익률', '수지비율', '인건비 대 총비용비율', 'R & D 투자효율', '총비용대비세금율', '금융비용 대 총비용비율', '감가상각비 대 총비용비율', '감가상각률', '누적감가상각률', '이자부담률', '지급이자율', '차입금평균이자율', '유보율', '사내유보율', '사내유보 대 자기자본비율', '적립금비율(재정비율)', '평균배당률', '자기자본배당률', '배당성향', '1주당매출액(원)', '1주당순이익(원)', '1주당 CASH FLOW(원)', '1주당순자산(원)', '1주당정상영업이익(원)', '유동자산구성비율', '재고자산 대 유동자산비율', '유동자산 대 비유동자산비율', '당좌자산구성비율', '비유동자산구성비율', '자기자본구성비율', '타인자본구성비율', '자기자본배율', '비유동비율', '비유동장기적합률', '유동비율', '당좌비율', '현금비율', '매출채권비율', '재고자산 대 순운전자본비율', '매출채권 대 매입채무비율

In [ ]:
# ------------------------------------------------------------
# 세 데이터셋(kosdaq_연결IFRS, kosdaq_IFRS, kosdaq_gaap)의
# 교집합(공통) 컬럼 이름 확인
# ------------------------------------------------------------
common_cols = (
    set(dfs_clean['kosdaq_연결IFRS'].columns)
    & set(dfs_clean['kosdaq_개별IFRS'].columns)
    & set(dfs_clean['kosdaq_gaap'].columns)
)

print(f"🔗 세 파일 공통 컬럼 수: {len(common_cols)}개")
# print("🔗 공통 컬럼 목록:")
# for col in sorted(common_cols):
#     print(" -", col)


🔗 세 파일 공통 컬럼 수: 132개


In [ ]:
# 1) 교집합 컬럼 리스트 -----------------------------------------------------------------
common_cols = [
    '회사명', '거래소코드', '회계년도', '총자본증가율', '유형자산증가율', '비유동자산증가율',
    '유동자산증가율', '재고자산증가율', '자기자본증가율', '매출액증가율', '영업이익증가율',
    '순이익증가율', '종업원1인당 부가가치증가율', '종업원수증가율', '종업원1인당 매출액증가율',
    '종업원1인당 인건비증가율', '매출액총이익률', '영업이익률(매출액)', '매출액순이익률',
    '총자본사업이익률', '총자본영업이익률', '총자본순이익률', '자기자본영업이익률',
    '자기자본순이익률', '경영자본영업이익률', '경영자본순이익률', '자본금영업이익률',
    '자본금순이익률', '매출원가 대 매출액비율', '영업비용대영업수익비율', '금융비용부담률',
    '외환이익 대 매출액비율', '광고선전비 대 매출액비율', '세전이익대비세금율(차감전)',
    '기업순이익률', '수지비율', '인건비 대 총비용비율', 'R & D 투자효율', '총비용대비세금율',
    '금융비용 대 총비용비율', '감가상각비 대 총비용비율', '감가상각률', '누적감가상각률',
    '이자부담률', '지급이자율', '차입금평균이자율', '유보율', '사내유보율',
    '사내유보 대 자기자본비율', '적립금비율(재정비율)', '평균배당률', '자기자본배당률',
    '배당성향', '1주당매출액(원)', '유동자산구성비율', '재고자산 대 유동자산비율',
    '유동자산 대 비유동자산비율', '당좌자산구성비율', '비유동자산구성비율',
    '자기자본구성비율', '타인자본구성비율', '자기자본배율', '비유동비율', '비유동장기적합률',
    '유동비율', '당좌비율', '현금비율', '매출채권비율', '재고자산 대 순운전자본비율',
    '매출채권 대 매입채무비율', '매출채권 대 상,제품비율', '매입채무 대 재고자산비율',
    '부채비율', '유동부채비율', '단기차입금 대 총차입금비율', '비유동부채비율',
    '비유동부채 대 순운전자본비율', '순운전자본비율', '차입금의존도', '차입금비율',
    '이자보상배율(이자비용)', '이자보상배율(순금융비용)', '유보액대비율',
    '유보액 대 납입자본배율', '유동자산집중도', '비유동자산집중도', '투자집중도',
    'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율', 'CASH FLOW 대 총자본비율',
    'CASH FLOW 대 매출액비율', '이자보상배율(영업이익)', '총자본회전률', '경영자본회전률',
    '자기자본회전률', '자본금회전률', '타인자본회전률', '매입채무회전률', '매입채무회전기간',
    '유동자산회전률', '당좌자산회전률', '재고자산회전률', '재고자산회전기간',
    '상품,제품회전률', '원,부재료회전률', '재공품회전률', '매출채권회전률', '매출채권회전기간',
    '비유동자산회전률', '유형자산회전율', '순운전자본회전률', '운전자본회전률', '1회전기간',
    '부가가치(백만원)', '종업원1인당 부가가치(백만원)', '종업원1인당 매출액(백만원)',
    '종업원당영업이익(백만원)', '종업원1인당 순이익(백만원)', '종업원1인당 인건비(백만원)',
    '노동장비율', '기계장비율', '자본집약도', '총자본투자효율', '설비투자효율', '기계투자효율',
    '부가가치율', '노동소득분배율', '자본분배율', '이윤분배율', '영업현금흐름(천원)',
    '투자활동으로 인한 현금흐름(*)(천원)', '재무활동으로 인한 현금흐름(*)(천원)'
]

# 2) 저장 경로 준비 ---------------------------------------------------------------------
save_dir = 'clean_columns_name2'
os.makedirs(save_dir, exist_ok=True)

# 3) 4개 DataFrame에서 공통 컬럼만 추출 후 Excel 저장 -----------------------------
for name in ['kosdaq_연결IFRS', 'kosdaq_개별IFRS', 'kosdaq_gaap', 'out_gaap_1','out_gaap_2','out_gaap_3','out_gaap_4']:
    df = dfs_clean[name]

    # 실제로 존재하는 컬럼만 선택 (예기치 않은 누락 대비)
    cols_present = [c for c in common_cols if c in df.columns]

    # 추출
    df_sub = df[cols_present]

    # Excel로 저장
    file_path = os.path.join(save_dir, f"{name}_common.xlsx")
    df_sub.to_excel(file_path, index=False)

    print(f"{name}: saved {len(cols_present)} cols → {file_path}")


kosdaq_연결IFRS: saved 132 cols → clean_columns_name2\kosdaq_연결IFRS_common.xlsx
kosdaq_개별IFRS: saved 132 cols → clean_columns_name2\kosdaq_개별IFRS_common.xlsx
kosdaq_gaap: saved 132 cols → clean_columns_name2\kosdaq_gaap_common.xlsx
out_gaap_1: saved 132 cols → clean_columns_name2\out_gaap_1_common.xlsx
out_gaap_2: saved 132 cols → clean_columns_name2\out_gaap_2_common.xlsx
out_gaap_3: saved 132 cols → clean_columns_name2\out_gaap_3_common.xlsx
out_gaap_4: saved 132 cols → clean_columns_name2\out_gaap_4_common.xlsx


In [ ]:
# 공통 컬럼 데이터 불러오기
clean_kos_연결 = pd.read_excel('./clean_columns_name2/kosdaq_연결IFRS_common.xlsx',dtype={'거래소코드':int})
clean_kos_개별 = pd.read_excel('./clean_columns_name2/kosdaq_개별IFRS_common.xlsx',dtype = {'거래소코드':int})
clean_kos_gaap = pd.read_excel('./clean_columns_name2/kosdaq_gaap_common.xlsx',dtype = {'거래소코드':int})
clean_out_gaap_1 =pd.read_excel('./clean_columns_name2/out_gaap_1_common.xlsx',dtype = {'거래소코드':int})
clean_out_gaap_2 =pd.read_excel('./clean_columns_name2/out_gaap_2_common.xlsx',dtype = {'거래소코드':int})
clean_out_gaap_3 =pd.read_excel('./clean_columns_name2/out_gaap_3_common.xlsx',dtype = {'거래소코드':int})
clean_out_gaap_4 =pd.read_excel('./clean_columns_name2/out_gaap_4_common.xlsx',dtype = {'거래소코드':int})

In [ ]:
kos_연결 = clean_kos_연결.copy()
kos_개별 = clean_kos_개별.copy()
kos_gaap = clean_kos_gaap.copy()
out_gaap_1 = clean_out_gaap_1.copy()
out_gaap_2 = clean_out_gaap_2.copy()
out_gaap_3 = clean_out_gaap_3.copy()
out_gaap_4 = clean_out_gaap_4.copy()

In [ ]:
# 각 데이터프레임별 회사 개수 확인
dfs = {
    'kosdaq_연결IFRS': kos_연결,
    'kosdaq_개별IFRS': kos_개별,
    'kosdaq_gaap': kos_gaap,
    'out_gaap_1': out_gaap_1,
    'out_gaap_2': out_gaap_2,
    'out_gaap_3': out_gaap_3,
    'out_gaap_4': out_gaap_4
}

for name, df in dfs.items():
    print(f'=== {name}의 회사 수 ===')
    # 3-1. 기본 정보: 행·열 개수, 컬럼별 non-null 값, 데이터 타입
    print(len(df['회사명'].unique()))

# 고유 회사 수 총 합계 개산
all_names_arr = (
    pd.concat([df['회사명'] for df in dfs.values()], ignore_index=True)  # 하나로 이어붙이기
      .dropna()                                                         # NaN 제거
      .unique()                                                         # 고유값만
)

print(f"총 고유 회사 수: {len(all_names_arr)}")

=== kosdaq_연결IFRS의 회사 수 ===
1936
=== kosdaq_개별IFRS의 회사 수 ===
1936
=== kosdaq_gaap의 회사 수 ===
1936
=== out_gaap_1의 회사 수 ===
19486
=== out_gaap_2의 회사 수 ===
19367
=== out_gaap_3의 회사 수 ===
19357
=== out_gaap_4의 회사 수 ===
3712


In [ ]:
# ――― ‘회사명-거래소코드’ 쌍을 한 곳에 모아 유니크화 ―――――――――――――――――――
all_name_code_df = (
    pd.concat(
        [
            df[['회사명', '거래소코드']]
              .dropna(subset=['회사명', '거래소코드'])   # 둘 다 값이 있을 때만
              for df in dfs.values()
              if '거래소코드' in df.columns             # 해당 컬럼이 있는 DF만
        ],
        ignore_index=True
    )
    .astype({'거래소코드': 'int'})                       # 타입 통일(중간 처리 편의를 위해)
    .drop_duplicates()                                  # 회사명+코드 완전 중복 제거
)

print(f"총 고유 ‘회사-코드’ 쌍: {len(all_name_code_df):,}")

총 고유 ‘회사-코드’ 쌍: 65,671


In [ ]:
# '회사명' 기준으로 그룹화해서 거래소코드가 2개 이상인 회사명만 필터
duplicated_code_names = (
    all_name_code_df
    .groupby('회사명')['거래소코드']
    .nunique()                # 거래소코드 개수 세기
    .reset_index()
    .query('거래소코드 > 1') # 거래소코드가 2개 이상인 회사명만
)

print(f"회사명은 같지만 거래소코드가 여러 개인 회사")
print(duplicated_code_names)

회사명은 같지만 거래소코드가 여러 개인 회사
             회사명  거래소코드
190       (주)가리온      3
203        (주)가야      2
249      (주)가화개발      2
385        (주)건우      3
387      (주)건우기업      2
...          ...    ...
63067   후림개발주식회사      2
63105    휴마시스(주)      2
63124      휴젤(주)      2
63153   흥진산업주식회사      2
63167  희망배움터주식회사      2

[2135 rows x 2 columns]


In [ ]:
filtered_df = all_name_code_df[all_name_code_df['회사명'] == '(주)가리온']
print(filtered_df)

          회사명   거래소코드
52966  (주)가리온   85096
52969  (주)가리온   95282
52974  (주)가리온  113874


In [ ]:
all_name_code_df = all_name_code_df.astype({'거래소코드': 'object'})  # 문자열형 고정

# 1) 회계연도 문자열 리스트 생성 ('2010/12' ~ '2023/12')
years = [f"{yr}/12" for yr in range(2010, 2024)]          # 14개

# 2) 회계연도 전용 DataFrame (14×1)
years_df = pd.DataFrame({'회계년도': years})

# 3) 카티션 곱(모든 회사 × 모든 연도)
expanded_df = (
    all_name_code_df.assign(_key=1)                 # 임시 열 추가
                  .merge(years_df.assign(_key=1), on='_key')  # _key로 교차(카티션) 조인
                  .drop('_key', axis=1)             # 임시 열 제거
)

# 4) 결과 확인
print(expanded_df.head(15))
print(f"전체 행 개수: {len(expanded_df):,} ")


         회사명  거래소코드     회계년도
0   (주)CMG제약  58820  2010/12
1   (주)CMG제약  58820  2011/12
2   (주)CMG제약  58820  2012/12
3   (주)CMG제약  58820  2013/12
4   (주)CMG제약  58820  2014/12
5   (주)CMG제약  58820  2015/12
6   (주)CMG제약  58820  2016/12
7   (주)CMG제약  58820  2017/12
8   (주)CMG제약  58820  2018/12
9   (주)CMG제약  58820  2019/12
10  (주)CMG제약  58820  2020/12
11  (주)CMG제약  58820  2021/12
12  (주)CMG제약  58820  2022/12
13  (주)CMG제약  58820  2023/12
14   (주)ES큐브  50120  2010/12
전체 행 개수: 919,394 


In [ ]:
expanded_df[expanded_df['회사명']=='(주)가리온'].head(20)

,회사명,거래소코드,회계년도
29484,(주)가리온,85096,2010/12
29485,(주)가리온,85096,2011/12
29486,(주)가리온,85096,2012/12
29487,(주)가리온,85096,2013/12
29488,(주)가리온,85096,2014/12
29489,(주)가리온,85096,2015/12
29490,(주)가리온,85096,2016/12
29491,(주)가리온,85096,2017/12
29492,(주)가리온,85096,2018/12
29493,(주)가리온,85096,2019/12


In [ ]:
expanded_df.to_csv('종합base.csv')

## 본격적인 재무 데이터 병합
- 외감 기업 재무 데이터 병합 - GAAP
- 코스닥 재무 데이터 병합 순서 - 1순위 : 연결 IFRS -> 2순위 : 개별 IFRS -> 3순위 : GAAP


In [ ]:
# 베이스 라인 파일 불러오기 ( 회사명, 거래소코드, 회계년도 )
base = pd.read_csv('종합base.csv')

In [ ]:
base.info()
base=base.iloc[:,1:]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835814 entries, 0 to 835813
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  835814 non-null  int64 
 1   회사명         835814 non-null  object
 2   거래소코드       835814 non-null  int64 
 3   회계년도        835814 non-null  object
dtypes: int64(2), object(2)
memory usage: 25.5+ MB


In [ ]:
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 835814 entries, 0 to 835813
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   회사명     835814 non-null  object
 1   거래소코드   835814 non-null  int64 
 2   회계년도    835814 non-null  object
dtypes: int64(1), object(2)
memory usage: 19.1+ MB


###코스닥

In [ ]:
# kos_연결에서 재무 데이터 컬럼 목록
financial_cols = kos_연결.columns.difference(['회사명', '거래소코드', '회계년도'])

# base에 kos_연결 재무 데이터 결합
base = (
    base.merge(
        kos_연결,                                   # 코스닥 연결 IFRS 재무 데이터
        on=['회사명', '거래소코드', '회계년도'],
        how='left'
    )
)

# 값이 추가된 경우 1, 그렇지 않으면 0 표시
base['연결_추가여부'] = (base[financial_cols].notna().any(axis=1)).astype(int)

In [ ]:
base['연결_추가여부'].value_counts()

연결_추가여부
0    823106
1     12708
Name: count, dtype: int64

In [ ]:
# kos_개별에서 재무 데이터 컬럼 목록
financial_cols = kos_개별.columns.difference(['회사명', '거래소코드', '회계년도'])

# 연결_추가여부가 0인 행 인덱스
mask = base['연결_추가여부'] == 0

# 해당 행들만 kos_개별 데이터 병합 후 재무 데이터 채우기
base.loc[mask, financial_cols] = (
    base.loc[mask, ['회사명', '거래소코드', '회계년도']]
        .merge(
            kos_개별,                                   # 코스닥 개별 IFRS 재무 데이터
            on=['회사명', '거래소코드', '회계년도'],
            how='left'
        )[financial_cols]
        .values
)

# 개별 추가 여부 컬럼 생성
base['개별_추가여부'] = 0
base.loc[mask, '개별_추가여부'] = (
    base.loc[mask, financial_cols].notna().any(axis=1)
).astype(int)


In [ ]:
base['개별_추가여부'].value_counts()

개별_추가여부
0    831826
1      3988
Name: count, dtype: int64

In [ ]:
# kos_gaap에서 재무 데이터 컬럼 목록
gaap_cols = kos_gaap.columns.difference(['회사명', '거래소코드', '회계년도'])

# 연결_추가여부와 개별_추가여부가 모두 0인 행
mask = (base['연결_추가여부'] == 0) & (base['개별_추가여부'] == 0)

# kos_gaap 데이터 병합 후 재무 데이터 채우기
base.loc[mask, gaap_cols] = (
    base.loc[mask, ['회사명', '거래소코드', '회계년도']]
        .merge(
            kos_gaap,
            on=['회사명', '거래소코드', '회계년도'],
            how='left'
        )[gaap_cols]
        .values
)

# kos_gaap 추가 여부 컬럼 생성
base['kos_gaap_추가여부'] = 0
base.loc[mask, 'kos_gaap_추가여부'] = (
    base.loc[mask, gaap_cols].notna().any(axis=1)
).astype(int)

In [ ]:
base['kos_gaap_추가여부'].value_counts()

kos_gaap_추가여부
0    835559
1       255
Name: count, dtype: int64

### 외감 기업

In [ ]:
# out_gaap에서 재무 데이터 컬럼 목록
out_gaap_1_cols = out_gaap_1.columns.difference(['회사명', '거래소코드', '회계년도'])

# 모든 추가여부 컬럼이 0인 행
mask = (
    (base['연결_추가여부'] == 0) &
    (base['개별_추가여부'] == 0) &
    (base['kos_gaap_추가여부'] == 0)
)

# out_gaap 데이터 병합 후 재무 데이터 채우기
base.loc[mask, out_gaap_1_cols] = (
    base.loc[mask, ['회사명', '거래소코드', '회계년도']]
        .merge(
            out_gaap_1,
            on=['회사명', '거래소코드', '회계년도'],
            how='left'
        )[out_gaap_1_cols]
        .values
)

# out_gaap 추가 여부 컬럼 생성
base['out_gaap_1_추가여부'] = 0
base.loc[mask, 'out_gaap_1_추가여부'] = (
    base.loc[mask, out_gaap_1_cols].notna().any(axis=1)
).astype(int)

In [ ]:
base['out_gaap_1_추가여부'].value_counts()

out_gaap_1_추가여부
0    729626
1    106188
Name: count, dtype: int64

In [ ]:
# out_gaap에서 재무 데이터 컬럼 목록
out_gaap_2_cols = out_gaap_2.columns.difference(['회사명', '거래소코드', '회계년도'])

# 모든 추가여부 컬럼이 0인 행
mask = (
    (base['연결_추가여부'] == 0) &
    (base['개별_추가여부'] == 0) &
    (base['kos_gaap_추가여부'] == 0)&
    (base['out_gaap_1_추가여부']==0)
)

# out_gaap 데이터 병합 후 재무 데이터 채우기
base.loc[mask, out_gaap_2_cols] = (
    base.loc[mask, ['회사명', '거래소코드', '회계년도']]
        .merge(
            out_gaap_2,
            on=['회사명', '거래소코드', '회계년도'],
            how='left'
        )[out_gaap_2_cols]
        .values
)

# out_gaap 추가 여부 컬럼 생성
base['out_gaap_2_추가여부'] = 0
base.loc[mask, 'out_gaap_2_추가여부'] = (
    base.loc[mask, out_gaap_2_cols].notna().any(axis=1)
).astype(int)


In [ ]:
base['out_gaap_2_추가여부'].value_counts()

out_gaap_2_추가여부
0    722251
1    113563
Name: count, dtype: int64

In [ ]:
# out_gaap에서 재무 데이터 컬럼 목록
out_gaap_3_cols = out_gaap_3.columns.difference(['회사명', '거래소코드', '회계년도'])

# 모든 추가여부 컬럼이 0인 행
mask = (
    (base['연결_추가여부'] == 0) &
    (base['개별_추가여부'] == 0) &
    (base['kos_gaap_추가여부'] == 0)&
    (base['out_gaap_1_추가여부']==0)&
    (base['out_gaap_2_추가여부']==0)
)

# out_gaap 데이터 병합 후 재무 데이터 채우기
base.loc[mask, out_gaap_3_cols] = (
    base.loc[mask, ['회사명', '거래소코드', '회계년도']]
        .merge(
            out_gaap_3,
            on=['회사명', '거래소코드', '회계년도'],
            how='left'
        )[out_gaap_3_cols]
        .values
)

# out_gaap 추가 여부 컬럼 생성
base['out_gaap_3_추가여부'] = 0
base.loc[mask, 'out_gaap_3_추가여부'] = (
    base.loc[mask, out_gaap_3_cols].notna().any(axis=1)
).astype(int)


In [ ]:
base['out_gaap_3_추가여부'].value_counts()

out_gaap_3_추가여부
0    726975
1    108839
Name: count, dtype: int64

In [ ]:
# out_gaap에서 재무 데이터 컬럼 목록
out_gaap_4_cols = out_gaap_4.columns.difference(['회사명', '거래소코드', '회계년도'])

# 모든 추가여부 컬럼이 0인 행
mask = (
    (base['연결_추가여부'] == 0) &
    (base['개별_추가여부'] == 0) &
    (base['kos_gaap_추가여부'] == 0)&
    (base['out_gaap_1_추가여부']==0)&
    (base['out_gaap_2_추가여부']==0)&
    (base['out_gaap_3_추가여부']==0)
)

# out_gaap 데이터 병합 후 재무 데이터 채우기
base.loc[mask, out_gaap_4_cols] = (
    base.loc[mask, ['회사명', '거래소코드', '회계년도']]
        .merge(
            out_gaap_4,
            on=['회사명', '거래소코드', '회계년도'],
            how='left'
        )[out_gaap_4_cols]
        .values
)

# out_gaap 추가 여부 컬럼 생성
base['out_gaap_4_추가여부'] = 0
base.loc[mask, 'out_gaap_4_추가여부'] = (
    base.loc[mask, out_gaap_4_cols].notna().any(axis=1)
).astype(int)


In [ ]:
base['out_gaap_4_추가여부'].value_counts()

out_gaap_4_추가여부
0    809576
1     26238
Name: count, dtype: int64

In [ ]:
flags = ['연결_추가여부', '개별_추가여부', 'kos_gaap_추가여부', 'out_gaap_1_추가여부', 'out_gaap_2_추가여부', 'out_gaap_3_추가여부', 'out_gaap_4_추가여부']

rows_with_any = base.loc[base[flags].any(axis=1)].sum()
print(f"추가여부가 하나라도 1인 행 개수: {rows_with_any}")

# ❷ 조건을 만족하는 행만 추출
matched_df = base.loc[base[flags].any(axis=1)]

# ❸ Excel로 저장
output_path = "이자비용처리 전 데이터.csv"   # 원하는 파일명·경로로 바꿔도 됨
matched_df.to_csv(output_path, index=False)

print(f"✅ 저장 완료: {output_path} (행 수={len(matched_df)})")

추가여부가 하나라도 1인 행 개수: 371779


### 비재무 데이터, 거시경제 데이터 수집하여 통합
- 비재무 데이터 수집 : TS2000
- 거시경제 데이터 : ECOS, FRED
- 통합 데이터 생성 : 총 컬럼 220개 (기준 컬럼 3개, 재무 132개, 비재무 및 거시경제 85개)

In [ ]:
# 파생위한종합 역시 이자비용처리 전 데이터와 같은방식으로 병합함(재무비율 대신 재무지표를 가져온 것 + 거시경제 데이터)

# 회계년도 컬럼 정수형으로 변경 및 추가여부 컬럼삭제
df = pd.read_csv("data/파생위한재무종합.csv",index_col=0)
df['회계년도'] = df['회계년도'].str.split('/').str[0]
df['회계년도'] = df['회계년도'].astype('int64')
df.drop(columns=['연결_추가여부', '개별_추가여부', 'kos_gaap_추가여부','비상장_연결_추가여부', '비상장_개별_추가여부', '비상장_gaap_추가여부'],inplace=True)

In [ ]:
# 데이터 조인하기
df1 = pd.read_csv('data/이자비용처리 전 데이터.csv',index_col=0)
join_data = df1.merge(df,how='left',on=['회사명','거래소코드','회계년도'])
join_data.to_csv('통합데이터.csv',index=False)

## 통합 데이터에 대한 간단한 EDA

In [ ]:
df = pd.read_csv("통합데이터.csv")
all_data = df.copy()

In [ ]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371776 entries, 0 to 371775
Columns: 221 entries, 회사명 to 부실여부
dtypes: float64(204), int64(13), object(4)
memory usage: 626.9+ MB


In [ ]:
all_data.describe()

,거래소코드,회계년도,총자본증가율,유형자산증가율,비유동자산증가율,유동자산증가율,재고자산증가율,자기자본증가율,매출액증가율,영업이익증가율,순이익증가율,종업원1인당 부가가치증가율,종업원수증가율,종업원1인당 매출액증가율,종업원1인당 인건비증가율,매출액총이익률,영업이익률(매출액),매출액순이익률,총자본사업이익률,총자본영업이익률,총자본순이익률,자기자본영업이익률,자기자본순이익률,경영자본영업이익률,경영자본순이익률,자본금영업이익률,자본금순이익률,매출원가 대 매출액비율,영업비용대영업수익비율,금융비용부담률,외환이익 대 매출액비율,광고선전비 대 매출액비율,세전이익대비세금율(차감전),기업순이익률,수지비율,인건비 대 총비용비율,R & D 투자효율,총비용대비세금율,금융비용 대 총비용비율,감가상각비 대 총비용비율,감가상각률,누적감가상각률,이자부담률,지급이자율,차입금평균이자율,유보율,사내유보율,사내유보 대 자기자본비율,적립금비율(재정비율),평균배당률,자기자본배당률,배당성향,1주당매출액(원),유동자산구성비율,재고자산 대 유동자산비율,유동자산 대 비유동자산비율,당좌자산구성비율,비유동자산구성비율,자기자본구성비율,타인자본구성비율,자기자본배율,비유동비율,비유동장기적합률,유동비율,당좌비율,현금비율,매출채권비율,재고자산 대 순운전자본비율,매출채권 대 매입채무비율,"매출채권 대 상,제품비율",매입채무 대 재고자산비율,부채비율,유동부채비율,단기차입금 대 총차입금비율,비유동부채비율,비유동부채 대 순운전자본비율,순운전자본비율,차입금의존도,차입금비율,이자보상배율(이자비용),이자보상배율(순금융비용),유보액대비율,유보액 대 납입자본배율,유동자산집중도,비유동자산집중도,투자집중도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,이자보상배율(영업이익),총자본회전률,경영자본회전률,자기자본회전률,자본금회전률,타인자본회전률,매입채무회전률,매입채무회전기간,유동자산회전률,당좌자산회전률,재고자산회전률,재고자산회전기간,"상품,제품회전률","원,부재료회전률",재공품회전률,매출채권회전률,매출채권회전기간,비유동자산회전률,유형자산회전율,순운전자본회전률,운전자본회전률,1회전기간,부가가치(백만원),종업원1인당 부가가치(백만원),종업원1인당 매출액(백만원),종업원당영업이익(백만원),종업원1인당 순이익(백만원),종업원1인당 인건비(백만원),노동장비율,기계장비율,자본집약도,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,영업현금흐름(천원),투자활동으로 인한 현금흐름(*)(천원),재무활동으로 인한 현금흐름(*)(천원),연결_추가여부,개별_추가여부,kos_gaap_추가여부,out_gaap_1_추가여부,out_gaap_2_추가여부,out_gaap_3_추가여부,out_gaap_4_추가여부,종업원,미국실업률,유로실업률,일본실업률,영국실업률,무역수지,미국소비자물가지수,소비자판매액지수,실질GDP,연준금리,GDP,소비자심리지수,미국비농업고용지수,정책금리,외국인투자동향(전체),Lf 평잔,경제심리지수,수입물량지수,수출물량지수,수입물가지수,수출물가지수,소비자물가지수,경기종합지수,경상수지,생산자물가지수,예금은행 총수신,외환보유액,spread_10y_3y,spread_10y_3m,credit_spread_AA,credit_spread_BBB,나스닥,유로화,파운드화,엔화,LNG,WTI유,가솔린,구리,난방유,납,니켈,대두,면,밀,설탕,쌀,알루미늄,옥수수,주석,천연가스,커피,코코아,금 수익률,돈육 수익률,백금 수익률,생우 수익률,육우 수익률,은 수익률,Nominal index,RTWEXBGS,VIX,US_spread_10y_3y,US_spread_10y_3m,US_credit_spread_AA,credit_spread_BBB.1,DOW JONES,NASDAQ,S&P500,kospi,kosdaq,G20,Korea,US,China,대표이사변경여부,수도권,업력,부실여부
count,371776.000000,371776.000000,371737.000000,3.717370e+05,3.717370e+05,3.717370e+05,3.717370e+05,3.717370e+05,3.717370e+05,3.717370e+05,3.717370e+05,359053.000000,359053.000000,359053.000000,359053.000000,371727.000000,3.717270e+05,3.717270e+05,371727.000000,371727.000000,371727.000000,3.717270e+05,3.717270e+05,3.717270e+05,3.717270e+05,3.717270e+05,3.717270e+05,371727.000000,3.717270e+05,3.717270e+05,371727.000000,3.717270e+05,3.717270e+05,371727.000000,3.717270e+05,371727.000000,371727.000000,371727.000000,3.717270e+05,371727.000000,371727.000000,3.717270e+05,3.717270e+05,3.717270e+05,3.717270e+05,3.717270e+05,359043.000000,3.590430e+05,3.717270e+05,3.590430e+05,359043.000000,3.590430e+05,3.717270e+05,371734.000000,371734.000000,3.717340e+05,371734.000000,371734.000000,371734.000000,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,371734.000000,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,3.717340e+05,371736.000000,371736.000000,3.717360e+05,3.717360e+05,371736.000000,3.717360e+05,3.717360e+05,371736.000000,371736.000000,3.717360e+05,3.717360e+05,3.717360e+05,3.717360e+05,3.717360e+05,3.717360e+05,3.717360e+05,3.717360e+05,3.717360e+05,371736.000000,371736.000000,371736.000000,3.717240e+05,359040.000000,359040.000000,156655.000000,359040.000000,359040.000000,359040.000000,359040.000000,359040.000000,3.717240e+05,3.717240e+05,3.717240e+05,3.717240e+05,3.717240e+05,3.717240e+05,3.717240e+05,3.717750e+05,3.717750e+05,3.717750e+05,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000,371776.000000

### 이자보상배율이 999999999인 행은 삭제
- 정상기업일 확률이 높기 때문에 데이터 불균형 문제를 해결하기 위해서 삭제하였음

In [ ]:
all_data = all_data[all_data['이자보상배율(이자비용)'] != 999999999]

### 데이터 라벨링
- 부실 : 1, 정상 0 라벨링
- 부실기업 정의 : 3년 연속 이자보상배율이 1 미만인 기업
- 3년 이상 재무 데이터가 있는 기업만 추출
- 부실 발생 시점 이후 행 삭제
    - ex) A회사 2012, 2013, 2014 이자보상배율 1 미만 -> 2014년 부실 판정, 2015년부터 행 삭제
- 2010,2011년은 라벨링을 위한 데이터이므로 삭제

In [ ]:
filtered = all_data.groupby(['회사명', '거래소코드']).filter(lambda g: len(g) >= 3)

In [ ]:
# 함수 정의 ─ 회사명 + 거래소코드 기준으로 부실여부 생성
def mark_default(df: pd.DataFrame,
                 group_cols: list = ["회사명", "거래소코드"],
                 year_col: str = "회계년도",
                 ics_col: str = "이자보상배율(이자비용)",
                 target: str = "부실여부") -> pd.DataFrame:
    """
    ▣ 로직
      1) 기업별(year_col 오름차순) 정렬
      2) 이자보상배율 < 1 이 3행 연속이면
           └ 세 번째(연속 완성) 행만 target = 1
           └ 그 이후 행은 제거
      3) 나머지는 target = 0
    """

    def _process(group: pd.DataFrame) -> pd.DataFrame:
        g = group.sort_values(year_col).copy()

        # ① <1 조건 시퀀스
        cond = g[ics_col].astype(float) < 1
        rolling3 = cond.rolling(window=3).sum()

        # ② 기본값 0, 첫 3연속 완성 행만 1
        g[target] = 0
        hits = np.flatnonzero(rolling3 == 3)
        if hits.size:
            pos = hits[0]
            g.iloc[pos, g.columns.get_loc(target)] = 1
            g = g.iloc[:pos + 1]
        return g

    # ③ 그룹별 적용 (회사명 + 거래소코드 기준)
    out = (
        df.groupby(group_cols, group_keys=False)
          .apply(_process)
    )
    return out

# 함수 실행 ─ filtered 에 새 컬럼 생성 & 부실 이후 행 제거
filtered = mark_default(filtered)

# 확인
print(filtered['부실여부'].value_counts(dropna=False))

In [ ]:
filtered = filtered[filtered['회계년도']>2011]

In [ ]:
filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 345529 entries, 0 to 371775
Columns: 220 entries, 회사명 to 업력
dtypes: float64(204), int64(12), object(4)
memory usage: 582.6+ MB


In [ ]:
# 이자보상배율 컬럼 제거
filtered.drop(columns=[ '이자보상배율(이자비용)',
 '이자보상배율(순금융비용)',
'이자보상배율(영업이익)'
],inplace=True)

### 범주형 변수 추가

In [ ]:
# 각 행마다 어떤 회계기준 재무데이터를 가져왔는지에 대한 컬럼 추가
filtered['GAAP'] = filtered['kos_gaap_추가여부'] + filtered['out_gaap_1_추가여부'] + filtered['out_gaap_2_추가여부'] + \
filtered['out_gaap_3_추가여부'] + filtered['out_gaap_4_추가여부']
filtered.rename(columns={'연결_추가여부' : 'IFRS_CONN','개별_추가여부' : 'IFRS'},inplace=True)

def year_check(x: pd.Series) -> int:
    return int(len(x) != (x.max() - x.min() + 1))

filtered['회계년도연속여부'] = filtered.groupby(['회사명', '거래소코드'])['회계년도'].transform(
    lambda x: year_check(x)
)
filtered['회계년도연속여부'] = filtered['회계년도연속여부'].astype('object')

# 추가여부 컬럼 삭제
drop_cols = ['kos_gaap_추가여부',
 'out_gaap_1_추가여부',
 'out_gaap_2_추가여부',
 'out_gaap_3_추가여부',
 'out_gaap_4_추가여부']
filtered.drop(columns=drop_cols,inplace=True)

# object 타입으로 변경
object_cols = [
'IFRS_CONN',
 'IFRS',
'대표이사변경여부',
 '수도권','부실여부','GAAP','회계년도연속여부']

for col in object_cols:
    filtered[col] = filtered[col].astype('object')

## 컬럼 이름 변경

In [ ]:
# 변화를 적용할 컬럼 리스트
indicators = [
    '미국실업률', '유로실업률', '일본실업률', '영국실업률', '무역수지',
    '미국소비자물가지수', '소비자판매액지수', '실질GDP', '연준금리', 'GDP',
    '소비자심리지수', '미국비농업고용지수', '정책금리', '외국인투자동향(전체)',
    'Lf 평잔', '경제심리지수', '수입물량지수', '수출물량지수', '수입물가지수',
    '수출물가지수', '소비자물가지수', '경기종합지수', '경상수지', '생산자물가지수',
    '예금은행 총수신', '외환보유액', 'spread_10y_3y', 'spread_10y_3m',
    'credit_spread_AA', 'credit_spread_BBB', '나스닥', '유로화', '파운드화',
    '엔화', 'LNG', 'WTI유', '가솔린', '구리', '난방유', '납', '니켈', '대두',
    '면', '밀', '설탕', '쌀', '알루미늄', '옥수수', '주석', '천연가스',
    '커피', '코코아', 'Nominal index', 'RTWEXBGS', 'VIX',
    'US_spread_10y_3y', 'US_spread_10y_3m', 'US_credit_spread_AA',
    'credit_spread_BBB.1', 'DOW JONES', 'NASDAQ', 'S&P500',
    'kospi', 'kosdaq', 'G20', 'Korea', 'US', 'China'
]

# ── 1) (원래이름 → 원래이름변화율) 매핑 만들기 ─────────────────────────
rename_map = {col: f"{col}변화율" for col in indicators}

# ── 2) 실제로 존재하는 컬럼만 추려서 rename ─────────────────────
# * 존재하지 않는 컬럼을 넘겨도 pandas는 무시하지만,
#   불필요한 매핑을 제거하면 가독성이 좋음
rename_map = {k: v for k, v in rename_map.items() if k in filtered.columns}

# in-place 변경: 결과를 같은 DataFrame에 반영
filtered.rename(columns=rename_map, inplace=True)

# ── 3) 실제로 변경된 컬럼 목록 출력 ─────────────────────────────
changed_columns = list(rename_map.values())
print("변경된 컬럼 목록:")
print(changed_columns)


변경된 컬럼 목록:
['미국실업률변화율', '유로실업률변화율', '일본실업률변화율', '영국실업률변화율', '무역수지변화율', '미국소비자물가지수변화율', '소비자판매액지수변화율', '실질GDP변화율', '연준금리변화율', 'GDP변화율', '소비자심리지수변화율', '미국비농업고용지수변화율', '정책금리변화율', '외국인투자동향(전체)변화율', 'Lf 평잔변화율', '경제심리지수변화율', '수입물량지수변화율', '수출물량지수변화율', '수입물가지수변화율', '수출물가지수변화율', '소비자물가지수변화율', '경기종합지수변화율', '경상수지변화율', '생산자물가지수변화율', '예금은행 총수신변화율', '외환보유액변화율', 'spread_10y_3y변화율', 'spread_10y_3m변화율', 'credit_spread_AA변화율', 'credit_spread_BBB변화율', '나스닥변화율', '유로화변화율', '파운드화변화율', '엔화변화율', 'LNG변화율', 'WTI유변화율', '가솔린변화율', '구리변화율', '난방유변화율', '납변화율', '니켈변화율', '대두변화율', '면변화율', '밀변화율', '설탕변화율', '쌀변화율', '알루미늄변화율', '옥수수변화율', '주석변화율', '천연가스변화율', '커피변화율', '코코아변화율', 'Nominal index변화율', 'RTWEXBGS변화율', 'VIX변화율', 'US_spread_10y_3y변화율', 'US_spread_10y_3m변화율', 'US_credit_spread_AA변화율', 'credit_spread_BBB.1변화율', 'DOW JONES변화율', 'NASDAQ변화율', 'S&P500변화율', 'kospi변화율', 'kosdaq변화율', 'G20변화율', 'Korea변화율', 'US변화율', 'China변화율']


In [ ]:
# 대상 컬럼 리스트
ch1 = ['G20변화율', 'Korea변화율', 'US변화율', 'China변화율']
ch2 = ['Nominal index변화율', 'RTWEXBGS변화율']

# 1) ch1 ─ '변화율' 바로 앞에 '경기선행지수' 삽입
rename_ch1 = {
    col: col.replace('변화율', '경기선행지수변화율')
    for col in ch1 if col in filtered.columns
}

# 2) ch2 ─ 영어 이름을 한글로 치환
rename_ch2 = {
    'Nominal index변화율': '명목 달러 인덱스 변화율',
    'RTWEXBGS변화율'    : '실질 달러 인덱스 변화율',
}
rename_ch2 = {k: v for k, v in rename_ch2.items() if k in filtered.columns}

# 3) 합쳐서 한 번에 적용 (in-place)
rename_map = {**rename_ch1, **rename_ch2}
filtered.rename(columns=rename_map, inplace=True)

# 4) 실제로 변경된 컬럼 목록 출력
changed_columns = list(rename_map.values())
print("변경된 컬럼 목록:")
print(changed_columns)

변경된 컬럼 목록:
['G20경기선행지수변화율', 'Korea경기선행지수변화율', 'US경기선행지수변화율', 'China경기선행지수변화율', '명목 달러 인덱스 변화율', '실질 달러 인덱스 변화율']


## 국면별 구분
- dickinson의 기업 생애주기 분류
- 도입기, 성장기, 쇠퇴기, 성숙기, (기타)

In [ ]:
op_col = '영업현금흐름(천원)'
inv_col = '투자활동으로 인한 현금흐름(*)(천원)'
fin_col = '재무활동으로 인한 현금흐름(*)(천원)'

# 국면 분류 함수 정의
def classify_dickinson(row):
    op = row[op_col]
    inv = row[inv_col]
    fin = row[fin_col]

    if op < 0 and inv < 0 and fin > 0:
        return '도입기'
    elif op > 0 and inv < 0 and fin > 0:
        return '성장기'
    elif op > 0 and inv < 0 and fin < 0:
        return '성숙기'
    elif op < 0 and inv > 0:
        return '쇠퇴기'
    else:
        return '기타'

# 데이터프레임에 생애주기 컬럼 추가
filtered['생애주기'] = filtered.apply(classify_dickinson, axis=1)

# 결과 확인
print(filtered['생애주기'].value_counts())
print("\n총 기업 수:", filtered['생애주기'].value_counts().sum())

생애주기
성숙기    75804
성장기    61331
도입기    47687
기타     45693
쇠퇴기    26841
Name: count, dtype: int64

총 기업 수: 257356


In [ ]:
# 생애주기 기타인 행 삭제
filtered_lifecycle = filtered[filtered['생애주기'] != '기타'].copy()


# 생애주기별 데이터 비율 (각 생애주기 / 전체 행)
counts = filtered_lifecycle['생애주기'].value_counts()          # 개수
ratio  = (counts / len(filtered_lifecycle)).rename('비율')      # 전체 대비 비율

# lifecycles 딕셔너리는 앞서 만든 그대로 사용한다고 가정
# lifecycles = {'초기': df1, '성장': df2, ...}

SAVE_DIR = "./생애주기별_CSV"          # 저장할 폴더(없으면 생성)
os.makedirs(SAVE_DIR, exist_ok=True)

lifecycles = {'도입' : filtered_lifecycle[filtered_lifecycle['생애주기']=='도입기'],
              '성장' : filtered_lifecycle[filtered_lifecycle['생애주기']=='성장기'],
              '성숙' : filtered_lifecycle[filtered_lifecycle['생애주기']=='성숙기'],
              '쇠퇴' : filtered_lifecycle[filtered_lifecycle['생애주기']=='쇠퇴기']}

for stage, df in lifecycles.items():
    # 파일 이름에 사용할 수 없는 문자를 제거하거나 _ 로 치환
    safe_stage = re.sub(r'[\\/:*?"<>| ]+', '_', str(stage))
    file_path = os.path.join(SAVE_DIR, f"eda{safe_stage}.csv")

    # CSV 저장 (BOM 포함 UTF-8 로 엑셀 호환성 확보)
    df.to_csv(file_path, index=False, encoding="utf-8-sig")

    print(f"✅ 저장 완료 → {file_path}  ({df.shape[0]} rows)")

생애주기
성숙기    75804
성장기    61331
도입기    47687
쇠퇴기    26841
Name: count, dtype: int64 


총 기업 수: 211663

          건수        비율
생애주기                 
성숙기   75804  0.358135
성장기   61331  0.289758
도입기   47687  0.225297
쇠퇴기   26841  0.126810


## 정리
- 코스닥, 외감 기업의 재무, 비재무, 거시경제 데이터 병합 및 전처리
- 부실 정의 : 3년 연속 이자보상배율이 1 미만인 기업
- 부실 여부 라벨링 후 dickinson의 국면별로 데이터셋 분리
    - 부실 비율 : 도입기 9.92%,, 성숙기 2.98%, 성장기 3.13%, 쇠퇴기 15.43%
- 파생변수 추가 및 EDA 후 모델링 진행 예정